In [1]:
import numpy as np
import pandas as pd
from CODECbreakCode.AudioMixer import FullTrackAudioMixer
import CODECbreakCode.Evaluator as Evaluator
from CODECbreakCode.Evaluator import MeasureHAAQIOutput
import argparse
from Optimiser.config import get_config, normalize_action, denormalize_action

In [2]:
Reggae_Mixing_Path = '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae'
Reggae_Noise_Generator_NC = FullTrackAudioMixer(Reggae_Mixing_Path)
#Noise_Generator_MP3.ManipulateInitGAIN([0, 0, 0, 0])
Reggae_Referece_File = Reggae_Noise_Generator_NC.TestDynNoisedFullTrack([0]*24,"Reference_IN_FULL.wav",isNormalised=False,isCompensated=True)
print(f"Referece_File:{Reggae_Referece_File}")

Vocal duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Drum duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Bass duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Other duration orginal is 17.462666666666667 seconds, now is the 8.0,  the audio changing to the MONO
Mixing File Load Sucessful
Referece_File:/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result/Reference_IN_FULL.wav


/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


In [3]:
Reggae_Referece_NCFile = Evaluator.NeuralCodecCompress(Reggae_Referece_File,24)
print(f"Reggae_Referece_AACFile:{Reggae_Referece_NCFile}")

Opt_fold: /home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result
Decoded audio: torch.Size([1, 2, 384000])
Reggae_Referece_AACFile:/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result/Mixing_Result_NeuralCodec_Wav/Reference_IN_FULL_24kbps.wav


In [4]:
####initalise the Haaqi
MeasureHAAQI = MeasureHAAQIOutput(Reggae_Referece_NCFile)#Initilize the HAAQI with a permanent reference
MeasureHAAQI.MeasureHAQQIOutput(Reggae_Referece_NCFile) #Test on how far from itself to itself

0.974981747709733

In [9]:
import tempfile
import os
BASE_DIR = "/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/"
TMP_SUBDIR = os.path.join(BASE_DIR, "tmp")

# make sure it exists once at startup:
os.makedirs(TMP_SUBDIR, exist_ok=True)
def haaqi_reward_muti_fn(solution: np.ndarray, is_normalised=True) -> float:
    if is_normalised:
        solution = denormalize_action(solution)
#    print(f'solution:{solution}, the type is {type(solution), type(solution[0])}')
 
    # Create a unique temp‐file name
    fd, degradated_filename = tempfile.mkstemp(prefix="dynC_", suffix=".wav")
    os.close(fd)  # we’ll let your compressor write to that path

    try:
        solution = list(solution)
        gener_Audio = Reggae_Noise_Generator_NC.TestDynNoisedFullTrack(solution,degradated_filename,isNormalised=False,isCompensated=True)
        gener_Audio_mp3 = Evaluator.NeuralCodecCompress(gener_Audio, 24)
        score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
    finally:
        # clean up
        try:
            os.remove(degradated_filename)
        except OSError:
            pass

    return round(1 - score, 4)

### PPO Trainning

In [6]:
import Optimiser

In [7]:
#import Optimiser.continous_RL_train_PPO
from Optimiser.continous_RL_train_PPO import continous_RL_train_PPO as CRLTrain

2025-08-10 16:19:54.917310: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-10 16:19:55.134436: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-10 16:19:55.134481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-10 16:19:55.166261: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-10 16:19:55.239610: I tensorflow/core/platform/cpu_feature_guar

In [10]:
trainner = CRLTrain(sub_episode_length=3, sub_episode_num_single_batch=3, env_num=1)
trainner.set_environments(haaqi_reward_muti_fn)
trainner.train(update_num=2, eval_intv=2)

number of sub_episodes used for a single param update: 3
train_env.batch_size = parallel environment number =  1
update_num: 2, eval_intv: 2


/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


Opt_fold: /home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result/tmp
Decoded audio: torch.Size([1, 2, 384000])


/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


Opt_fold: /home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result/tmp
Decoded audio: torch.Size([1, 2, 384000])


...

Opt_fold: /home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result/tmp
Decoded audio: torch.Size([1, 2, 384000])


/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


Opt_fold: /home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result/tmp
Decoded audio: torch.Size([1, 2, 384000])


/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


Opt_fold: /home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result/tmp
Decoded audio: torch.Size([1, 2, 384000])
Epoch 1: policy_loss=-0.118  value_loss=3.163  entropy_loss=-0.022  clip_fraction=0.667
final_solution= [ 60.     2.29 -22.78   1.14   1.   257.64  35.     3.     0.     1.95
   1.5  228.79  35.     3.   -25.     4.8    1.   200.    60.     3.
 -12.59   5.     1.   200.  ] final_reward= 0.5797


In [12]:
solution =[60.,2.29,-22.78,1.14,1.,257.64,35.,3.,0.,1.95,1.5,228.79,35.,3.,-25.,4.8, 1.,200., 60.,3.,-12.59,5.,1.,200.,]
haaqi_reward_muti_fn(solution, is_normalised=False)

/root/anaconda3/envs/codecbreak/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


Opt_fold: /home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result/tmp
Decoded audio: torch.Size([1, 2, 384000])


0.5741

In [13]:
Reggae_Noise_Generator_NC.EraseTheMixing()

Cleared all contents in '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result'.
Cleared all contents in '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result_Mp3'.
Cleared all contents in '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result_Mp3_Wav'.
Cleared all contents in '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result_AAC'.
Cleared all contents in '/home/codecrack/CodecBreakerwithRL/AudioEX/Reggae/Mixing_Result_AAC_Wav'.
